In [188]:
import pandas as pd
import numpy as np

file = pd.read_csv("individual_Recode.csv", low_memory=False)

clust_admin_1 = pd.read_csv("cluster_to_admin_1.csv")
clust_admin_2 = pd.read_csv("cluster_to_admin_2.csv")
clust_admin_3 = pd.read_csv("cluster_to_admin_3.csv")

# Rename columns in preparation for joining.
clust_admin_1 = clust_admin_1.rename(columns = {'DHSCLUST':'V001'})
clust_admin_2 = clust_admin_2.rename(columns = {'DHSCLUST':'V001'})
clust_admin_3 = clust_admin_3.rename(columns = {'DHSCLUST':'V001'})

# Join the dataframes. 
c1 = pd.merge(file, clust_admin_1, how='inner', on='V001')
c2 = pd.merge(c1, clust_admin_2, how='inner', on='V001')
c3 = pd.merge(c2, clust_admin_3, how='inner', on='V001')

keep_columns = ['V001','V002','V005','M14$1', 'ID_1', 'ID_2', 'ID_3']
c3 = c3 [keep_columns]

file.to_csv("health_access_data.csv", index=False)
c3.to_csv("complete_health_access_data.csv", index=False)

In [189]:
def compute_simple_wha_aggregate(region_name, col_name, data, cluster_file):
    admin_data = pd.DataFrame(columns=[region_name, 'Total_Participants', 'No_Antenatal_Care', 'Median_Antenatal_Visits'])
    admin_data[region_name] = list(cluster_file[col_name].unique())
    
    # Set index to region numbers.   
    wha_data = data
    wha_data.set_index(col_name, inplace=True)
    
    # Set index to region numbers and sort index.
    admin_data.set_index(region_name, inplace=True)
    admin_data.fillna(0, inplace=True)
    admin_data.sort_index(ascending=True, inplace=True)
    
    wha_data['M14$1'] = pd.to_numeric(wha_data['M14$1'], errors='coerce').fillna(-1).astype(int)
    
    for index, row in wha_data.iterrows():
        result = row['M14$1']
        
        #  Increment the number of participants.
        participants = admin_data.loc[index, 'Total_Participants'] + 1
        admin_data.loc[index, 'Total_Participants'] = participants

        #  Increment the number of participants who have not had any antenatal care.
        if(result == 0):
            no_care = admin_data.loc[index, 'No_Antenatal_Care'] + 1
            admin_data.loc[index, 'No_Antenatal_Care'] = no_care       
            
    return admin_data

In [190]:
admin_1 = compute_simple_wha_aggregate("Admin_1_Region", "ID_1", c3, clust_admin_1)
admin_2 = compute_simple_wha_aggregate("Admin_2_Region", "ID_2", c3, clust_admin_2)
admin_3 = compute_simple_wha_aggregate("Admin_3_Region", "ID_3", c3, clust_admin_3)

admin_1.to_csv("wha_aggregate_1.csv", index=True)
admin_2.to_csv("wha_aggregate_2.csv", index=True)
admin_3.to_csv("wha_aggregate_3.csv", index=True)

# NEED TO ADD MEDIAN NUMBER OF ANTENATAL CARE VISITS FOR EACH REGION